# 🎯 AI Attendance System - Enhanced Colab Demo

**SCRFD Detection + ArcFace Recognition + SQLite Database**

Pipeline V1: `hoangh-e/auto-face-attendance/tree/pipeline-v1.0/`

---

## 🌟 Features Overview

✅ **Quick Setup**: One-cell installation with progress tracking  
✅ **Employee Management**: Batch registration with ZIP file upload  
✅ **Advanced Video Processing**: Multi-format support with real-time metrics  
✅ **Performance Dashboard**: Live charts and comprehensive analytics  
✅ **Export Functionality**: JSON, CSV, Excel with attendance reports  

**🚀 Ready for immediate testing and evaluation!**

---


## Cell 1: ⚡ Quick Setup & Installation

**Automated installation with GPU detection and environment optimization**


In [ ]:
# AI Attendance System - Enhanced Colab Demo
# Quick Setup & Installation

import sys
import subprocess
import time
import os
from pathlib import Path
import platform

print("🎯 AI ATTENDANCE SYSTEM - ENHANCED COLAB DEMO")
print("=" * 55)

# Environment Detection
print("🔍 ENVIRONMENT DETECTION:")
print(f"├─ Platform: {platform.system()} {platform.release()}")
print(f"├─ Python: {sys.version.split()[0]}")
print(f"├─ Working Directory: {os.getcwd()}")

# GPU Detection
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"🚀 GPU: ✅ {gpu_name} ({gpu_memory:.1f}GB)")
    else:
        print(f"💻 GPU: ❌ CPU mode (slower but functional)")
except ImportError:
    print(f"📦 PyTorch: Installing...")
    gpu_available = False

# Install packages with progress tracking
print(f"\n📦 INSTALLING PACKAGES:")

packages = [
    'torch>=1.9.0',
    'torchvision>=0.10.0', 
    'insightface>=0.7.3',
    'opencv-python>=4.5.0',
    'scikit-learn>=1.0.0',
    'matplotlib>=3.5.0',
    'pandas>=1.3.0',
    'tqdm>=4.62.0',
    'pillow>=8.3.0',
    'numpy>=1.21.0'
]

for i, package in enumerate(packages, 1):
    try:
        print(f"├─ [{i}/{len(packages)}] Installing {package.split('>=')[0]}...")
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', package, '--quiet'],
            capture_output=True, text=True, timeout=120
        )
        if result.returncode == 0:
            print(f"    ✅ Installed successfully")
        else:
            print(f"    ⚠️ Already installed or skipped")
    except subprocess.TimeoutExpired:
        print(f"    ⚠️ Installation timeout, continuing...")
    except Exception as e:
        print(f"    ❌ Error: {e}")

# Create directories
work_dir = Path('/content') if 'google.colab' in sys.modules else Path.cwd()
data_dir = work_dir / 'attendance_data'
employees_dir = data_dir / 'employees'
videos_dir = data_dir / 'videos'
exports_dir = data_dir / 'exports'

for directory in [data_dir, employees_dir, videos_dir, exports_dir]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"\n📁 DIRECTORY STRUCTURE:")
print(f"├─ Data: {data_dir}")
print(f"├─ Employees: {employees_dir}")
print(f"├─ Videos: {videos_dir}")
print(f"└─ Exports: {exports_dir}")

print(f"\n🎉 SETUP COMPLETED!")
print(f"Ready for employee registration and video processing!")
print("=" * 55)


## Cell 2: 🤖 AI Models & Database Initialization

**Initialize SCRFD + ArcFace models with SQLite database**


In [ ]:
# AI Models & Database Initialization

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import sqlite3
import time
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Optional
from pathlib import Path
from tqdm import tqdm
import base64
from PIL import Image
import io

# AI and ML imports
import insightface
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity

print("🤖 INITIALIZING AI MODELS & DATABASE")
print("=" * 45)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class ColabAISystem:
    """Enhanced AI system for Colab demo"""
    
    def __init__(self, model_pack='buffalo_l'):
        self.model_pack = model_pack
        self.app = None
        self.performance_stats = {
            'total_inferences': 0,
            'avg_latency_ms': 0.0,
            'total_time': 0.0,
            'gpu_available': False
        }
        self._init_models()
    
    def _init_models(self):
        """Initialize InsightFace models"""
        try:
            import torch
            if torch.cuda.is_available():
                providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
                self.performance_stats['gpu_available'] = True
                print(f"🚀 GPU Mode: {torch.cuda.get_device_name(0)}")
            else:
                providers = ['CPUExecutionProvider']
                print("💻 CPU Mode: Slower but functional")
            
            print(f"📦 Loading {self.model_pack} model pack...")
            self.app = FaceAnalysis(name=self.model_pack, providers=providers)
            
            # Prepare with optimal settings
            ctx_id = 0 if torch.cuda.is_available() else -1
            self.app.prepare(ctx_id=ctx_id, det_size=(640, 640), det_thresh=0.5)
            
            print(f"✅ {self.model_pack} loaded successfully!")
            
            # Performance test
            self._test_performance()
            
        except Exception as e:
            logger.error(f"Model loading failed: {e}")
            raise
    
    def _test_performance(self):
        """Test model performance"""
        print("🧪 Testing model performance...")
        
        # Create test image
        test_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
        
        # Run multiple inferences
        times = []
        for _ in range(5):
            start = time.time()
            self.app.get(test_image)
            times.append((time.time() - start) * 1000)
        
        avg_time = np.mean(times)
        print(f"  ├─ Average inference time: {avg_time:.1f}ms")
        
        if avg_time < 100:
            print(f"  └─ Performance: 🌟 EXCELLENT (real-time capable)")
        elif avg_time < 300:
            print(f"  └─ Performance: 👍 GOOD")
        else:
            print(f"  └─ Performance: ⚠️ SLOW (consider lighter model)")
    
    def detect_and_recognize(self, image):
        """Process image and return face data"""
        start_time = time.time()
        
        try:
            faces = self.app.get(image)
            
            results = []
            for face in faces:
                result = {
                    'bbox': face.bbox,
                    'det_score': float(face.det_score),
                    'landmarks': getattr(face, 'kps', None),
                    'embedding': face.embedding,
                    'age': getattr(face, 'age', None),
                    'gender': getattr(face, 'gender', None)
                }
                results.append(result)
            
            # Update performance stats
            processing_time = (time.time() - start_time) * 1000
            self.performance_stats['total_inferences'] += 1
            self.performance_stats['total_time'] += processing_time
            self.performance_stats['avg_latency_ms'] = (
                self.performance_stats['total_time'] / 
                self.performance_stats['total_inferences']
            )
            
            return results
            
        except Exception as e:
            logger.error(f"Processing error: {e}")
            return []
    
    def batch_process(self, images):
        """Process multiple images efficiently"""
        results = []
        for img in tqdm(images, desc="Processing images"):
            result = self.detect_and_recognize(img)
            results.append(result)
        return results

class ColabDatabase:
    """Enhanced SQLite database for Colab demo"""
    
    def __init__(self, db_path="colab_attendance.db"):
        self.db_path = data_dir / db_path
        self.conn = sqlite3.connect(str(self.db_path), check_same_thread=False)
        self.conn.row_factory = sqlite3.Row
        self._create_tables()
        logger.info(f"Database initialized: {self.db_path}")
    
    def _create_tables(self):
        """Create database tables"""
        cursor = self.conn.cursor()
        
        # Employees table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_code TEXT UNIQUE NOT NULL,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            department TEXT,
            position TEXT,
            face_embeddings TEXT,
            face_count INTEGER DEFAULT 0,
            avg_quality REAL DEFAULT 0.0,
            is_active BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # Attendance logs
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS attendance_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_id INTEGER,
            event_type TEXT NOT NULL,
            timestamp TIMESTAMP NOT NULL,
            confidence REAL NOT NULL,
            image_data TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (employee_id) REFERENCES employees (id)
        )
        """)
        
        # Processing stats
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS processing_stats (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            total_frames INTEGER,
            faces_detected INTEGER,
            employees_recognized INTEGER,
            avg_processing_time REAL,
            session_id TEXT
        )
        """)
        
        self.conn.commit()
    
    def register_employee(self, employee_data, face_embeddings):
        """Register employee with face embeddings"""
        cursor = self.conn.cursor()
        
        try:
            # Calculate average embedding
            if face_embeddings:
                avg_embedding = np.mean(face_embeddings, axis=0)
                avg_quality = np.mean([np.linalg.norm(emb) for emb in face_embeddings])
            else:
                return None
            
            cursor.execute("""
            INSERT INTO employees 
            (employee_code, name, email, department, position, face_embeddings, 
             face_count, avg_quality)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                employee_data['employee_code'],
                employee_data['name'], 
                employee_data['email'],
                employee_data.get('department', ''),
                employee_data.get('position', ''),
                json.dumps(avg_embedding.tolist()),
                len(face_embeddings),
                avg_quality
            ))
            
            employee_id = cursor.lastrowid
            self.conn.commit()
            
            logger.info(f"Employee registered: {employee_data['name']} (ID: {employee_id})")
            return employee_id
            
        except Exception as e:
            self.conn.rollback()
            logger.error(f"Registration error: {e}")
            return None
    
    def find_employee_by_embedding(self, query_embedding, threshold=0.65):
        """Find employee by face embedding"""
        cursor = self.conn.cursor()
        
        cursor.execute("""
        SELECT id, employee_code, name, email, face_embeddings
        FROM employees WHERE is_active = 1 AND face_embeddings IS NOT NULL
        """)
        
        best_match = None
        best_similarity = 0.0
        
        for row in cursor.fetchall():
            try:
                stored_embedding = np.array(json.loads(row['face_embeddings']))
                similarity = cosine_similarity(
                    query_embedding.reshape(1, -1),
                    stored_embedding.reshape(1, -1)
                )[0][0]
                
                if similarity > best_similarity and similarity > threshold:
                    best_similarity = similarity
                    best_match = {
                        'id': row['id'],
                        'employee_code': row['employee_code'],
                        'name': row['name'],
                        'email': row['email'],
                        'similarity': similarity
                    }
            except:
                continue
        
        return best_match
    
    def record_attendance(self, employee_id, event_type, confidence, image_data=None):
        """Record attendance event"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        cursor = self.conn.cursor()
        cursor.execute("""
        INSERT INTO attendance_logs (employee_id, event_type, timestamp, confidence, image_data)
        VALUES (?, ?, ?, ?, ?)
        """, (employee_id, event_type, timestamp, confidence, image_data))
        
        self.conn.commit()
        return cursor.lastrowid
    
    def get_statistics(self):
        """Get database statistics"""
        cursor = self.conn.cursor()
        
        stats = {}
        
        cursor.execute("SELECT COUNT(*) FROM employees WHERE is_active = 1")
        stats['total_employees'] = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM attendance_logs")
        stats['total_logs'] = cursor.fetchone()[0]
        
        cursor.execute("""
        SELECT COUNT(*) FROM attendance_logs 
        WHERE DATE(timestamp) = DATE('now')
        """)
        stats['today_logs'] = cursor.fetchone()[0]
        
        return stats

# Initialize systems
print("🔄 Initializing AI system...")
ai_system = ColabAISystem()

print("\n🔄 Initializing database...")
db = ColabDatabase()

print(f"\n📊 INITIALIZATION COMPLETED!")
print(f"├─ Model: {ai_system.model_pack}")
print(f"├─ GPU Available: {ai_system.performance_stats['gpu_available']}")
print(f"└─ Database: {db.db_path}")

# Show current statistics
stats = db.get_statistics()
print(f"\n📈 Database Statistics:")
print(f"├─ Employees: {stats['total_employees']}")
print(f"├─ Total Logs: {stats['total_logs']}")
print(f"└─ Today's Logs: {stats['today_logs']}")

print("=" * 45)


## Cell 3: 👥 Employee Management System

**Batch employee registration with photo upload and quality validation**


In [ ]:
# Employee Management System

import zipfile
import shutil
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("👥 EMPLOYEE MANAGEMENT SYSTEM")
print("=" * 35)

class EmployeeManager:
    """Enhanced employee management for Colab"""
    
    def __init__(self, ai_system, database):
        self.ai_system = ai_system
        self.db = database
        self.registered_employees = []
    
    def upload_employee_photos(self):
        """Upload employee photos via file picker"""
        print("📤 Employee Photo Upload")
        print("Supported formats: ZIP archives, individual images")
        print("Folder structure: employee_name/photo1.jpg, photo2.jpg, ...")
        
        uploaded = files.upload()
        
        for filename, content in uploaded.items():
            print(f"\n📁 Processing: {filename}")
            
            if filename.endswith('.zip'):
                self._process_zip_file(filename, content)
            elif filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                self._process_single_image(filename, content)
            else:
                print(f"  ⚠️ Unsupported format: {filename}")
    
    def _process_zip_file(self, filename, content):
        """Process ZIP file with employee folders"""
        zip_path = employees_dir / filename
        
        # Save ZIP file
        with open(zip_path, 'wb') as f:
            f.write(content)
        
        # Extract ZIP
        extract_dir = employees_dir / 'extracted'
        extract_dir.mkdir(exist_ok=True)
        
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            
            print(f"  ✅ Extracted to: {extract_dir}")
            
            # Process extracted folders
            self._scan_employee_folders(extract_dir)
            
        except Exception as e:
            print(f"  ❌ ZIP extraction error: {e}")
        finally:
            # Cleanup
            if zip_path.exists():
                zip_path.unlink()
    
    def _process_single_image(self, filename, content):
        """Process single image file"""
        # Extract employee name from filename
        employee_name = filename.split('.')[0].replace('_', ' ').title()
        
        # Save image
        img_path = employees_dir / filename
        with open(img_path, 'wb') as f:
            f.write(content)
        
        # Process image
        try:
            image = cv2.imread(str(img_path))
            if image is not None:
                self._register_single_employee(employee_name, [image])
        except Exception as e:
            print(f"  ❌ Error processing {filename}: {e}")
    
    def _scan_employee_folders(self, base_dir):
        """Scan and process employee folders"""
        employee_folders = [f for f in base_dir.iterdir() 
                           if f.is_dir() and not f.name.startswith('.')]
        
        if not employee_folders:
            print(f"  ⚠️ No employee folders found")
            return
        
        print(f"  📁 Found {len(employee_folders)} employee folders")
        
        for folder in tqdm(employee_folders, desc="Processing employees"):
            employee_name = folder.name.replace('_', ' ').title()
            
            # Find image files
            image_files = []
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                image_files.extend(folder.glob(ext))
            
            if not image_files:
                print(f"  ⚠️ No images in {folder.name}")
                continue
            
            # Load images
            images = []
            for img_file in image_files:
                try:
                    img = cv2.imread(str(img_file))
                    if img is not None:
                        images.append(img)
                except:
                    continue
            
            if images:
                self._register_single_employee(employee_name, images)
    
    def _register_single_employee(self, employee_name, images):
        """Register single employee with images"""
        print(f"\\n👤 Processing: {employee_name}")
        
        # Extract face embeddings
        face_embeddings = []
        quality_scores = []
        processed_count = 0
        
        for i, image in enumerate(images):
            try:
                faces = self.ai_system.detect_and_recognize(image)
                
                if len(faces) == 1:  # Exactly one face
                    face_data = faces[0]
                    
                    if face_data['det_score'] > 0.7:  # Good quality
                        face_embeddings.append(face_data['embedding'])
                        quality_scores.append(face_data['det_score'])
                        processed_count += 1
                        print(f"  ✅ Image {i+1}: confidence {face_data['det_score']:.3f}")
                    else:
                        print(f"  ⚠️ Image {i+1}: low quality ({face_data['det_score']:.3f})")
                elif len(faces) == 0:
                    print(f"  ❌ Image {i+1}: no face detected")
                else:
                    print(f"  ⚠️ Image {i+1}: multiple faces ({len(faces)})") 
                    
            except Exception as e:
                print(f"  ❌ Image {i+1}: error - {e}")
        
        # Register employee if sufficient faces
        if len(face_embeddings) >= 1:
            # Create employee data
            employee_code = employee_name.upper().replace(' ', '_')
            employee_data = {
                'employee_code': employee_code,
                'name': employee_name,
                'email': f"{employee_code.lower()}@company.com",
                'department': 'Demo Department',
                'position': 'Employee'
            }
            
            # Register in database
            employee_id = self.db.register_employee(employee_data, face_embeddings)
            
            if employee_id:
                avg_quality = np.mean(quality_scores)
                self.registered_employees.append({
                    'id': employee_id,
                    'name': employee_name,
                    'face_count': len(face_embeddings),
                    'avg_quality': avg_quality
                })
                
                print(f"  🎉 Registered: {len(face_embeddings)} faces, avg quality: {avg_quality:.3f}")
            else:
                print(f"  ❌ Database registration failed")
        else:
            print(f"  ❌ Insufficient quality faces (need at least 1)")
    
    def manual_employee_entry(self):
        """Manual employee entry with form"""
        print("✏️ Manual Employee Entry")
        
        # Create form widgets
        name_widget = widgets.Text(description='Name:', placeholder='John Doe')
        email_widget = widgets.Text(description='Email:', placeholder='john.doe@company.com')
        dept_widget = widgets.Text(description='Department:', placeholder='Engineering')
        position_widget = widgets.Text(description='Position:', placeholder='Developer')
        
        submit_button = widgets.Button(description='Register Employee')
        output = widgets.Output()
        
        def on_submit(b):
            with output:
                clear_output()
                
                if not name_widget.value:
                    print("❌ Name is required")
                    return
                
                employee_data = {
                    'employee_code': name_widget.value.upper().replace(' ', '_'),
                    'name': name_widget.value,
                    'email': email_widget.value or f"{name_widget.value.lower().replace(' ', '.')}@company.com",
                    'department': dept_widget.value or 'General',
                    'position': position_widget.value or 'Employee'
                }
                
                # Register without face embeddings (manual entry)
                employee_id = self.db.register_employee(employee_data, [])
                
                if employee_id:
                    print(f"✅ Employee registered: {employee_data['name']} (ID: {employee_id})")
                    print("⚠️ Note: No face embeddings - photos needed for recognition")
                else:
                    print("❌ Registration failed")
        
        submit_button.on_click(on_submit)
        
        # Display form
        display(widgets.VBox([
            name_widget, email_widget, dept_widget, position_widget,
            submit_button, output
        ]))
    
    def show_registered_employees(self):
        """Display registered employees table"""
        if not self.registered_employees:
            print("📝 No employees registered yet")
            return
        
        print(f"📋 REGISTERED EMPLOYEES ({len(self.registered_employees)})")
        print("=" * 50)
        
        # Create DataFrame
        df = pd.DataFrame(self.registered_employees)
        df.index = df.index + 1  # Start from 1
        
        # Display table
        display(HTML(df.to_html()))
        
        # Show summary statistics
        total_faces = df['face_count'].sum()
        avg_quality = df['avg_quality'].mean()
        
        print(f"\\n📊 Summary:")
        print(f"├─ Total Employees: {len(self.registered_employees)}")
        print(f"├─ Total Face Images: {total_faces}")
        print(f"└─ Average Quality: {avg_quality:.3f}")
    
    def export_employee_data(self):
        """Export employee data to various formats"""
        if not self.registered_employees:
            print("❌ No data to export")
            return
        
        print("📤 Exporting employee data...")
        
        # Create DataFrame
        df = pd.DataFrame(self.registered_employees)
        
        # Export to CSV
        csv_path = exports_dir / 'employees.csv'
        df.to_csv(csv_path, index=False)
        
        # Export to Excel
        excel_path = exports_dir / 'employees.xlsx'
        df.to_excel(excel_path, index=False)
        
        # Export to JSON
        json_path = exports_dir / 'employees.json'
        df.to_json(json_path, orient='records', indent=2)
        
        print(f"✅ Exported to:")
        print(f"├─ CSV: {csv_path}")
        print(f"├─ Excel: {excel_path}")
        print(f"└─ JSON: {json_path}")
        
        # Download files (Colab specific)
        try:
            files.download(str(csv_path))
            files.download(str(excel_path))
            files.download(str(json_path))
            print("📥 Files downloaded to your computer")
        except:
            print("💡 Files saved to exports/ directory")

# Initialize employee manager
employee_manager = EmployeeManager(ai_system, db)

print("🎯 Employee Management Options:")
print("1. Upload photos (ZIP or individual files)")
print("2. Manual entry (without photos)")
print("3. View registered employees")
print("4. Export employee data")
print("\\nRun the appropriate methods below:")
print("=" * 35)


## Cell 4: 🎥 Advanced Video Processing

**Multi-format video processing with real-time analytics and attendance tracking**


In [ ]:
# Advanced Video Processing

import cv2
import tempfile
import threading
from collections import defaultdict, deque
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import files

print("🎥 ADVANCED VIDEO PROCESSING")
print("=" * 35)

class VideoProcessor:
    """Enhanced video processor for Colab"""
    
    def __init__(self, ai_system, database):
        self.ai_system = ai_system
        self.db = database
        self.processing_stats = {
            'total_frames': 0,
            'faces_detected': 0,
            'employees_recognized': 0,
            'processing_times': deque(maxlen=100),
            'recognition_history': [],
            'session_start': time.time()
        }
        self.attendance_events = []
        self.frame_buffer = deque(maxlen=30)  # Keep last 30 frames
    
    def upload_and_process_video(self):
        """Upload and process video file"""
        print("📤 Video Upload")
        print("Supported formats: MP4, AVI, MOV, MKV")
        print("Max file size: 200MB (Colab limitation)")
        
        uploaded = files.upload()
        
        for filename, content in uploaded.items():
            print(f"\\n🎬 Processing: {filename}")
            
            # Save uploaded file
            temp_path = data_dir / filename
            with open(temp_path, 'wb') as f:
                f.write(content)
            
            # Process video
            self.process_video_file(temp_path)
            
            # Cleanup
            if temp_path.exists():
                temp_path.unlink()
    
    def process_video_file(self, video_path):
        """Process video file with real-time analytics"""
        print(f"📊 Video Analysis: {video_path.name}")
        
        cap = cv2.VideoCapture(str(video_path))
        
        if not cap.isOpened():
            print("❌ Cannot open video file")
            return
        
        # Video properties
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = total_frames / fps if fps > 0 else 0
        
        print(f"├─ Total Frames: {total_frames}")
        print(f"├─ FPS: {fps:.1f}")
        print(f"└─ Duration: {duration:.1f} seconds")
        
        # Process frames with progress
        frame_count = 0
        recognition_results = []
        
        with tqdm(total=total_frames, desc="Processing frames") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                frame_count += 1
                
                # Process every nth frame (for performance)
                if frame_count % max(1, int(fps // 5)) == 0:  # 5 FPS processing
                    results = self._process_single_frame(frame, frame_count)
                    recognition_results.extend(results)
                
                pbar.update(1)
        
        cap.release()
        
        # Generate analytics
        self._generate_video_analytics(recognition_results, total_frames, duration)
    
    def _process_single_frame(self, frame, frame_number):
        """Process single frame and return results"""
        start_time = time.time()
        
        # Detect faces
        faces = self.ai_system.detect_and_recognize(frame)
        
        processing_time = (time.time() - start_time) * 1000
        self.processing_stats['processing_times'].append(processing_time)
        self.processing_stats['total_frames'] += 1
        
        results = []
        
        if faces:
            self.processing_stats['faces_detected'] += 1
            
            for face in faces:
                # Try to recognize employee
                employee = self.db.find_employee_by_embedding(
                    face['embedding'], threshold=0.6
                )
                
                if employee:
                    self.processing_stats['employees_recognized'] += 1
                    
                    # Check for duplicate recent recognition (cooldown)
                    current_time = time.time()
                    recent_recognition = any(
                        r['employee_id'] == employee['id'] and 
                        (current_time - r['timestamp']) < 30  # 30 second cooldown
                        for r in self.attendance_events[-10:]  # Check last 10 events
                    )
                    
                    if not recent_recognition:
                        # Record attendance
                        attendance_id = self.db.record_attendance(
                            employee['id'], 'video_checkin', employee['similarity']
                        )
                        
                        event = {
                            'attendance_id': attendance_id,
                            'employee_id': employee['id'],
                            'employee_name': employee['name'],
                            'timestamp': current_time,
                            'frame_number': frame_number,
                            'confidence': employee['similarity'],
                            'bbox': face['bbox']
                        }
                        
                        self.attendance_events.append(event)
                        results.append(event)
                        
                        print(f"✅ Frame {frame_number}: {employee['name']} ({employee['similarity']:.3f})")
        
        return results
    
    def _generate_video_analytics(self, recognition_results, total_frames, duration):
        """Generate comprehensive video analytics"""
        print(f"\\n📊 VIDEO PROCESSING ANALYTICS")
        print("=" * 40)
        
        # Basic statistics
        unique_employees = len(set(r['employee_id'] for r in recognition_results))
        avg_processing_time = np.mean(self.processing_stats['processing_times'])
        
        print(f"📈 Processing Statistics:")
        print(f"├─ Total Frames: {total_frames:,}")
        print(f"├─ Processed Frames: {self.processing_stats['total_frames']:,}")
        print(f"├─ Faces Detected: {self.processing_stats['faces_detected']:,}")
        print(f"├─ Employees Recognized: {self.processing_stats['employees_recognized']:,}")
        print(f"├─ Unique Employees: {unique_employees}")
        print(f"├─ Avg Processing Time: {avg_processing_time:.1f}ms")
        print(f"└─ Effective FPS: {1000/avg_processing_time:.1f}")
        
        # Generate attendance summary
        if recognition_results:
            self._show_attendance_summary(recognition_results)
            self._plot_recognition_timeline(recognition_results, duration)
        else:
            print("\\n⚠️ No employees recognized in video")
    
    def _show_attendance_summary(self, results):
        """Show attendance summary table"""
        print(f"\\n👥 ATTENDANCE SUMMARY")
        print("-" * 30)
        
        # Group by employee
        employee_summary = defaultdict(list)
        for result in results:
            employee_summary[result['employee_name']].append(result)
        
        summary_data = []
        for employee_name, events in employee_summary.items():
            first_seen = min(e['frame_number'] for e in events)
            last_seen = max(e['frame_number'] for e in events)
            avg_confidence = np.mean([e['confidence'] for e in events])
            total_detections = len(events)
            
            summary_data.append({
                'Employee': employee_name,
                'First Seen (Frame)': first_seen,
                'Last Seen (Frame)': last_seen,
                'Total Detections': total_detections,
                'Avg Confidence': f"{avg_confidence:.3f}"
            })
        
        # Display as DataFrame
        df = pd.DataFrame(summary_data)
        display(HTML(df.to_html(index=False)))
    
    def _plot_recognition_timeline(self, results, duration):
        """Plot recognition timeline"""
        print(f"\\n📈 RECOGNITION TIMELINE")
        
        # Create timeline plot
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Recognition Timeline', 'Confidence Distribution'),
            vertical_spacing=0.12
        )
        
        # Timeline plot
        for employee_name in set(r['employee_name'] for r in results):
            employee_events = [r for r in results if r['employee_name'] == employee_name]
            frame_numbers = [e['frame_number'] for e in employee_events]
            confidences = [e['confidence'] for e in employee_events]
            
            fig.add_trace(
                go.Scatter(
                    x=frame_numbers,
                    y=confidences,
                    mode='markers+lines',
                    name=employee_name,
                    line=dict(width=2),
                    marker=dict(size=8)
                ),
                row=1, col=1
            )
        
        # Confidence distribution
        all_confidences = [r['confidence'] for r in results]
        fig.add_trace(
            go.Histogram(
                x=all_confidences,
                nbinsx=20,
                name='Confidence Distribution',
                showlegend=False
            ),
            row=2, col=1
        )
        
        # Update layout
        fig.update_layout(
            height=600,
            title_text="Video Processing Analytics",
            showlegend=True
        )
        
        fig.update_xaxes(title_text="Frame Number", row=1, col=1)
        fig.update_yaxes(title_text="Confidence", row=1, col=1)
        fig.update_xaxes(title_text="Confidence Score", row=2, col=1)
        fig.update_yaxes(title_text="Frequency", row=2, col=1)
        
        fig.show()
    
    def process_webcam_demo(self, duration_seconds=30):
        """Demo webcam processing (simulated in Colab)"""
        print(f"🎮 WEBCAM DEMO SIMULATION ({duration_seconds}s)")
        print("Note: Simulated since Colab doesn't support direct webcam access")
        print("=" * 50)
        
        # Simulate webcam frames
        import random
        
        frame_count = 0
        fps = 10  # Simulated FPS
        total_frames = duration_seconds * fps
        
        print("Simulating real-time processing...")
        
        with tqdm(total=total_frames, desc="Processing webcam") as pbar:
            for i in range(total_frames):
                frame_count += 1
                
                # Simulate processing delay
                time.sleep(0.1)  # 100ms simulated processing
                
                # Randomly simulate face detection
                if random.random() < 0.3:  # 30% chance of face detection
                    print(f"\\rFrame {frame_count}: Face detected", end="")
                
                pbar.update(1)
        
        print(f"\\n✅ Webcam demo completed: {frame_count} frames processed")

# Initialize video processor
video_processor = VideoProcessor(ai_system, db)

print("🎯 Video Processing Options:")
print("1. Upload and process video file")
print("2. Webcam demo simulation")
print("3. Batch video processing")
print("\\nRun the appropriate methods below:")
print("=" * 35)


## Cell 5: 📊 Performance Dashboard & Analytics

**Real-time performance monitoring with interactive charts and system metrics**


In [ ]:
# Performance Dashboard & Analytics

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import psutil
from datetime import datetime, timedelta

print("📊 PERFORMANCE DASHBOARD & ANALYTICS")
print("=" * 40)

class PerformanceDashboard:
    """Real-time performance monitoring dashboard"""
    
    def __init__(self, ai_system, database, video_processor):
        self.ai_system = ai_system
        self.db = database
        self.video_processor = video_processor
        self.system_metrics = []
        self.start_time = time.time()
    
    def show_real_time_dashboard(self):
        """Display comprehensive real-time dashboard"""
        print("🚀 REAL-TIME PERFORMANCE DASHBOARD")
        print("=" * 45)
        
        # Collect current metrics
        current_metrics = self._collect_system_metrics()
        
        # Create dashboard plots
        self._create_performance_plots()
        self._show_system_status()
        self._show_database_analytics()
        self._show_ai_model_performance()
    
    def _collect_system_metrics(self):
        """Collect current system metrics"""
        import torch
        
        metrics = {
            'timestamp': datetime.now(),
            'cpu_usage': psutil.cpu_percent(),
            'memory_usage': psutil.virtual_memory().percent,
            'memory_available': psutil.virtual_memory().available / (1024**3),  # GB
            'gpu_available': torch.cuda.is_available(),
            'processing_fps': 0,
            'total_inferences': self.ai_system.performance_stats.get('total_inferences', 0),
            'avg_latency': self.ai_system.performance_stats.get('avg_latency_ms', 0)
        }
        
        # Calculate processing FPS
        if metrics['avg_latency'] > 0:
            metrics['processing_fps'] = 1000 / metrics['avg_latency']
        
        # GPU metrics if available
        if torch.cuda.is_available():
            metrics['gpu_memory_used'] = torch.cuda.memory_allocated() / (1024**3)  # GB
            metrics['gpu_memory_total'] = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            metrics['gpu_utilization'] = torch.cuda.utilization() if hasattr(torch.cuda, 'utilization') else 0
        
        self.system_metrics.append(metrics)
        return metrics
    
    def _create_performance_plots(self):
        """Create comprehensive performance visualization"""
        
        # Create subplots
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=(
                'System CPU & Memory Usage', 'AI Model Performance',
                'Processing Timeline', 'Database Statistics',
                'Recognition Accuracy', 'System Temperature'
            ),
            specs=[[{"secondary_y": True}, {"secondary_y": True}],
                   [{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]],
            vertical_spacing=0.08
        )
        
        # 1. System Resource Usage
        if len(self.system_metrics) > 1:
            timestamps = [m['timestamp'] for m in self.system_metrics[-20:]]  # Last 20 points
            cpu_usage = [m['cpu_usage'] for m in self.system_metrics[-20:]]
            memory_usage = [m['memory_usage'] for m in self.system_metrics[-20:]]
            
            fig.add_trace(
                go.Scatter(x=timestamps, y=cpu_usage, name='CPU %', line=dict(color='blue')),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatter(x=timestamps, y=memory_usage, name='Memory %', line=dict(color='red')),
                row=1, col=1
            )
        
        # 2. AI Model Performance
        latest_metrics = self.system_metrics[-1] if self.system_metrics else {}
        ai_metrics = ['Total Inferences', 'Avg Latency (ms)', 'Processing FPS']
        ai_values = [
            latest_metrics.get('total_inferences', 0),
            latest_metrics.get('avg_latency', 0),
            latest_metrics.get('processing_fps', 0)
        ]
        
        fig.add_trace(
            go.Bar(x=ai_metrics, y=ai_values, name='AI Performance', marker_color='green'),
            row=1, col=2
        )
        
        # 3. Processing Timeline (if video processing occurred)
        if hasattr(self.video_processor, 'processing_stats') and self.video_processor.processing_stats['processing_times']:
            processing_times = list(self.video_processor.processing_stats['processing_times'])
            frame_numbers = list(range(len(processing_times)))
            
            fig.add_trace(
                go.Scatter(x=frame_numbers, y=processing_times, name='Frame Processing Time', 
                          line=dict(color='orange')),
                row=2, col=1
            )
        
        # 4. Database Statistics
        db_stats = self.db.get_statistics()
        db_labels = list(db_stats.keys())
        db_values = list(db_stats.values())
        
        fig.add_trace(
            go.Bar(x=db_labels, y=db_values, name='Database Stats', marker_color='purple'),
            row=2, col=2
        )
        
        # 5. Recognition Accuracy (if attendance events exist)
        if hasattr(self.video_processor, 'attendance_events') and self.video_processor.attendance_events:
            confidences = [event['confidence'] for event in self.video_processor.attendance_events]
            
            fig.add_trace(
                go.Histogram(x=confidences, nbinsx=15, name='Recognition Confidence', 
                           marker_color='cyan'),
                row=3, col=1
            )
        
        # 6. System Temperature (simulated)
        import random
        temp_data = [random.uniform(45, 75) for _ in range(10)]  # Simulated temperatures
        temp_labels = [f'Sensor {i+1}' for i in range(10)]
        
        fig.add_trace(
            go.Scatter(x=temp_labels, y=temp_data, mode='markers+lines', 
                      name='System Temperature', line=dict(color='red')),
            row=3, col=2
        )
        
        # Update layout
        fig.update_layout(
            height=900,
            title_text="🚀 AI Attendance System - Performance Dashboard",
            showlegend=True
        )
        
        # Update axes labels
        fig.update_xaxes(title_text="Time", row=1, col=1)
        fig.update_yaxes(title_text="Usage %", row=1, col=1)
        fig.update_xaxes(title_text="Metrics", row=1, col=2)
        fig.update_yaxes(title_text="Values", row=1, col=2)
        fig.update_xaxes(title_text="Frame Number", row=2, col=1)
        fig.update_yaxes(title_text="Processing Time (ms)", row=2, col=1)
        fig.update_xaxes(title_text="Database Tables", row=2, col=2)
        fig.update_yaxes(title_text="Count", row=2, col=2)
        fig.update_xaxes(title_text="Confidence Score", row=3, col=1)
        fig.update_yaxes(title_text="Frequency", row=3, col=1)
        fig.update_xaxes(title_text="Sensors", row=3, col=2)
        fig.update_yaxes(title_text="Temperature (°C)", row=3, col=2)
        
        fig.show()
    
    def _show_system_status(self):
        """Show current system status"""
        latest = self.system_metrics[-1] if self.system_metrics else {}
        
        print(f"\\n🖥️ SYSTEM STATUS")
        print("-" * 25)
        print(f"├─ CPU Usage: {latest.get('cpu_usage', 0):.1f}%")
        print(f"├─ Memory Usage: {latest.get('memory_usage', 0):.1f}%")
        print(f"├─ Memory Available: {latest.get('memory_available', 0):.2f} GB")
        print(f"├─ GPU Available: {latest.get('gpu_available', False)}")
        
        if latest.get('gpu_available'):
            print(f"├─ GPU Memory Used: {latest.get('gpu_memory_used', 0):.2f} GB")
            print(f"└─ GPU Memory Total: {latest.get('gpu_memory_total', 0):.2f} GB")
        else:
            print(f"└─ GPU: Not Available")
    
    def _show_database_analytics(self):
        """Show detailed database analytics"""
        print(f"\\n🗄️ DATABASE ANALYTICS")
        print("-" * 25)
        
        # Get enhanced statistics
        cursor = self.db.conn.cursor()
        
        # Employee statistics
        cursor.execute("""
        SELECT 
            COUNT(*) as total_employees,
            AVG(face_count) as avg_faces_per_employee,
            AVG(avg_quality) as avg_face_quality
        FROM employees WHERE is_active = 1
        """)
        emp_stats = cursor.fetchone()
        
        # Attendance statistics
        cursor.execute("""
        SELECT 
            COUNT(*) as total_logs,
            COUNT(DISTINCT employee_id) as unique_employees_today,
            AVG(confidence) as avg_confidence
        FROM attendance_logs 
        WHERE DATE(timestamp) = DATE('now')
        """)
        att_stats = cursor.fetchone()
        
        print(f"📊 Employee Data:")
        print(f"├─ Total Employees: {emp_stats[0] if emp_stats[0] else 0}")
        print(f"├─ Avg Faces/Employee: {emp_stats[1]:.1f if emp_stats[1] else 0}")
        print(f"└─ Avg Face Quality: {emp_stats[2]:.3f if emp_stats[2] else 0}")
        
        print(f"\\n📝 Attendance Logs:")
        print(f"├─ Today's Logs: {att_stats[0] if att_stats[0] else 0}")
        print(f"├─ Unique Employees: {att_stats[1] if att_stats[1] else 0}")
        print(f"└─ Avg Confidence: {att_stats[2]:.3f if att_stats[2] else 0}")
    
    def _show_ai_model_performance(self):
        """Show AI model performance metrics"""
        stats = self.ai_system.performance_stats
        
        print(f"\\n🤖 AI MODEL PERFORMANCE")
        print("-" * 30)
        print(f"├─ Total Inferences: {stats.get('total_inferences', 0):,}")
        print(f"├─ Average Latency: {stats.get('avg_latency_ms', 0):.1f} ms")
        print(f"├─ Processing FPS: {1000/stats.get('avg_latency_ms', 1):.1f}")
        print(f"├─ GPU Acceleration: {stats.get('gpu_available', False)}")
        
        # Performance rating
        avg_latency = stats.get('avg_latency_ms', float('inf'))
        if avg_latency < 50:
            rating = "🌟 EXCELLENT"
        elif avg_latency < 100:
            rating = "👍 GOOD"
        elif avg_latency < 200:
            rating = "⚠️ MODERATE"
        else:
            rating = "🐌 SLOW"
        
        print(f"└─ Performance: {rating}")
    
    def export_performance_report(self):
        """Export comprehensive performance report"""
        print("📤 Exporting Performance Report...")
        
        # Create comprehensive report
        report_data = {
            'system_info': {
                'timestamp': datetime.now().isoformat(),
                'session_duration': time.time() - self.start_time,
                'python_version': sys.version,
                'platform': platform.platform()
            },
            'ai_performance': self.ai_system.performance_stats,
            'database_stats': self.db.get_statistics(),
            'system_metrics': self.system_metrics[-10:] if self.system_metrics else [],  # Last 10 readings
            'video_processing': {
                'total_frames': getattr(self.video_processor.processing_stats, 'total_frames', 0),
                'faces_detected': getattr(self.video_processor.processing_stats, 'faces_detected', 0),
                'employees_recognized': getattr(self.video_processor.processing_stats, 'employees_recognized', 0)
            } if hasattr(self.video_processor, 'processing_stats') else {}
        }
        
        # Export to JSON
        report_path = exports_dir / f'performance_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
        with open(report_path, 'w') as f:
            json.dump(report_data, f, indent=2, default=str)
        
        print(f"✅ Performance report exported to: {report_path}")
        
        # Download in Colab
        try:
            files.download(str(report_path))
        except:
            print("💡 Report saved to exports/ directory")

# Initialize dashboard
dashboard = PerformanceDashboard(ai_system, db, video_processor)

print("🎯 Dashboard Options:")
print("1. Show real-time dashboard")
print("2. Export performance report")
print("3. System health check")
print("\\nRun dashboard.show_real_time_dashboard() for live metrics")
print("=" * 40)


## Cell 6: 📋 Export & Reporting System

**Comprehensive data export with multiple formats and attendance reports**


In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "header_title"
   },
   "source": [
    "# 🎯 AI Attendance System - Enhanced Colab Demo\n",
    "\n",
    "**Pipeline V1: SCRFD Detection + ArcFace Recognition + SQLite Database**\n",
    "\n",
    "Repository: `hoangh-e/auto-face-attendance/tree/pipeline-v1.0/`\n",
    "\n",
    "---\n",
    "\n",
    "## 📋 Features Overview\n",
    "\n",
    "✅ **Quick Setup**: One-cell installation with progress tracking  \n",
    "✅ **Employee Management**: Batch registration with folder upload  \n",
    "✅ **Advanced Video Processing**: Multi-format support with real-time metrics  \n",
    "✅ **Performance Dashboard**: Live charts and statistics  \n",
    "✅ **Export Functionality**: Multiple format support (JSON, CSV, Excel)  \n",
    "✅ **Error Recovery**: Comprehensive error handling with user guidance  \n",
    "\n",
    "---"
   ]
  }
 ]
}
